In [1]:
import pandas as pd
import numpy as np
import random 
from random import sample
from itertools import accumulate
from pandas.tseries.offsets import DateOffset

In [24]:
# N个零件,共5种类型
Job_num = 5
Ma_num = 3
job = [ np.random.randint(5,10)for i in range(Job_num)]
# 每种零件的加工时间
W_t = [ round(10*np.random.random(),0) for i in range(Job_num)]
# 每种零件的交付时间
D_t = [ t*10 for t in W_t]
# 每种零件对应的设备machine
machine = [ list(np.random.randint(2, size=Job_num)) for m in range(Ma_num)]
# 零件切换时间矩阵
Change_t = [[ np.random.randint(3) for i in range(Job_num)] for i in range(Job_num)]
# 加工效率最大化 ： 任务完工时间耗时 最小

In [3]:
def get_init(Job_num,job):
    init_x = sum([ [i]*job[i] for i in range(Job_num)],list())
#     print(init_x)
    random.shuffle(init_x)
#     print(init_x)
    return init_x

In [10]:
def get_fit(x):
    sol = [[] for i in range(Ma_num)]
    for id_,job in enumerate(x):
        for m in range(3):
            if id_%3 == m:
                sol[m].append(job)
#     print(sol)
    duration = [[] for i in range(Ma_num)]
    for m in range(Ma_num):
        for s in sol[m]:
            if machine[m][s] ==0:
                dt = 100
                duration[m].append(dt)
            else:
                duration[m].append(W_t[s])
    #get_duration
    fit = max([ list(accumulate(d))for d in duration][-1])
    return fit,duration,sol

In [20]:
import plotly.figure_factory as ff

colors = dict(Job_0 = 'rgb(46, 137, 205)',
              Job_1 = 'rgb(114, 44, 121)',
              Job_2 = 'rgb(198, 47, 105)',
              Job_3 = 'rgb(58, 149, 136)',
              Job_4 = 'rgb(107, 127, 135)')

In [30]:
def get_change(x):
    i,j = sample(range(len(x)),2)
    while x[i]==x[j]:
        i,j = sample(range(len(x)),2)
    x[i], x[j] = x[j], x[i]
    return x

# 局部搜索
def search(Job_num,job,max_no_improv=30):
    best = {}
    best['vector'] = get_init(Job_num,job)
    print(best['vector'])
    best['cost'] = get_fit(best['vector'])[0]
    iter_ = 0
    while iter_ < max_no_improv:
        new_x = get_change(best['vector'])
        if get_fit(new_x)[0] < best['cost']:
            best['vector'] = new_x
        iter_  = iter_+1
    return best

In [28]:
best = search(Job_num,job,30000)
print(best['cost'])

[1, 1, 0, 0, 0, 2, 2, 2, 4, 4, 4, 3, 4, 4, 1, 3, 4, 1, 2, 2, 2, 0, 2, 1, 0, 1, 0, 0, 1, 3, 0, 3, 1, 2, 4, 1, 3]
340.0


In [26]:
def get_plot(best):
    fit,duration,sol = get_fit(best['vector'])
    tmp = []
    for m in range(Ma_num):
        time_1 = [pd.Timestamp('2020-01-01 01') + DateOffset(hours = h) for h in list(accumulate(duration[m]))]
        for id_,time in enumerate(time_1[:-1]):
            tmp.append(dict(Task="Machine_{}".format(m),Start=str(time),
                            Finish=str(time_1[id_+1]),Job_id= 'Job_'+str(sol[m][id_])))
    fig = ff.create_gantt(tmp, colors=colors, index_col='Job_id', show_colorbar=True,
                      group_tasks=True)
    fig.show()

In [29]:
get_plot(best)

In [ ]:
# 